In [1]:
#| default_exp er.ma_model

In [2]:
#| hide
%load_ext autoreload
%autoreload 2

# Moving average model

To illustrate a backtesting algorithm, a simple prediction algorithm is presented. Aside from the simplicity of the prediction algorithm, it's assumed that predictions are made at the same frequency as the input data.

To create an expected return (ER) model (forecasting model), data and the exact data format are required. Here, both data and param are assumed to be dictionaries, facilitating the development of complex algorithms that require various data items.

The convention is that any ER will accept data and a param, and will consistently generate an output dictionary with a "yHat" element and possibly others that can be utilized in more complex algorithms (e.g., when creating allocation or risk).


In [3]:
#| export
import pandas as pd
import numpy as np

In [4]:
#| export

class ma_model:
    def __init__(self, data, param):
        """
        Initialize the moving average (ma) class with data and parameters.
        
        Parameters:
        - data: A DataFrame containing OHLCV (Open, High, Low, Close, Volume) data.
        - param: A dictionary containing parameters 'mawin' (moving average window) and 'sdwin' (standard deviation window).
        """
        self.mawin = param['mawin']  # Moving average window size
        self.sdwin = param['sdwin']  # Standard deviation window size
        self.ohlcv = data  # OHLCAV data: array

    def y_hat(self):
        """
        Calculate the yHat value, which is a normalized measure of the recent returns.
        
        Returns:
        - A single value representing the normalized recent returns.
        """
        # Calculate the log returns of the adjusted close prices
        r = np.diff(np.log(self.ohlcv[:,4]))
        
        # Calculate the mean of the recent returns over the moving average window
        mu = np.mean(r[-self.mawin:])
        
        # Calculate the standard deviation of the recent returns over the standard deviation window
        sig = np.std(r[-self.sdwin:])
        
        # Return the hyperbolic tangent of the ratio of mean to standard deviation
        # in order to keep the values between +/-1
        out ={}
        out['y_hat']    = float(np.round(np.tanh(mu / sig),4))
        return out

## Example

In [5]:
#| eval: false

# Get data
from backtest_sample.data_interface import get_data
import_module = get_data()
df = import_module('BTC-USD').sim_data()
# Forecast
prm = {}
prm['mawin'] = 10
prm['sdwin'] = 30
yHat = ma_model(df.values,prm)
print(yHat.y_hat())

[*********************100%***********************]  1 of 1 completed

{'y_hat': 0.4312}
{'y_hat': 0.4312}
